# RAG - Conversando com os seus dados

In [4]:
from dotenv import load_dotenv
load_dotenv()

from langchain.llms import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct')

C:\Users\annaf\AppData\Local\Temp\ipykernel_8532\1267886439.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model='gpt-3.5-turbo-instruct')


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
caminhos = [
    'assets/Explorando o Universo das IAs com Hugging Face.pdf',
    'assets/Explorando a API da OpenAI.pdf'
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())
    
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=['\n\n', '\n', '.', ' ', '']
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('assets/', '')
    doc.metadata['doc_id'] = i

In [7]:
diretorio = 'docs/chat_retrieval'

embeddings_model = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

## Criando estrutura de conversa

In [8]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [9]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectorstore.as_retriever(search_type='mmr')
)

In [10]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'

chat_chain.invoke({'query':pergunta})

{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma empresa que começou em 2017 na França, inicialmente focada no desenvolvimento de Chatbots. Atualmente, a empresa oferece uma plataforma que disponibiliza modelos de Inteligência Artificial, conjuntos de dados e aplicativos (chamados de "Spaces"). Para acessar a plataforma do Hugging Face, é necessário criar uma conta no site da empresa e, em seguida, criar um token de acesso para autenticação. A partir daí, é possível explorar e utilizar os modelos e recursos disponíveis na plataforma.'}

### Modificando o prompt da chain

In [11]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
"""Utilize o contexto fornceido para responder a pergunta ao final. 
Se você não sabe a resposta, apenas diga que não sabe e não tente inventar a resposta.
Utilize três frases no máximo, mantenha a resposta concisa.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectorstore.as_retriever(search_type='mmr'),
    chain_type_kwargs={'prompt':chain_prompt},
    return_source_documents = True
)

In [13]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'

resposta = chat_chain.invoke({'query':pergunta})
print(resposta['result'])

Hugging Face é uma empresa francesa que desenvolveu chatbots em 2017. Para acessá-lo, é necessário criar uma conta e solicitar autorização para uso de modelos restritos.


In [14]:
print(resposta['source_documents'])

[Document(metadata={'doc_id': 24, 'page': 5, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}, page_content='Explorando o Universo das IAs com Hugging Face\n01. O que é Hugging Face?\nBem-vindos ao curso de Hugging Face da Asimov Academy!\nNeste curso, vamos explorar as principais utilidades da plataforma de IAHugging Face. Aprenderemos\ncomo utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face\naos nossos scripts de Python.\nAfinal, o que é Hugging Face?\nA Hugging Face é uma empresa que iniciou em 2017 na França, com o desenvolvimento de Chatbots.'), Document(metadata={'doc_id': 28, 'page': 6, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}, page_content='O Hugging Face cobra apenas se você quiser utilizar a infraestrutura deles para hospedar algum\nprojeto privado pessoal ou da sua empresa.\nComo usaremos o Hugging Face?\nNeste curso, vamos explorar todo o potencial que existe na plataforma do Hugging Face. Isso i

## Outros tipos de chains

### Stuff

In [15]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectorstore.as_retriever(search_type='mmr'),
    chain_type='stuff'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'

resposta = chat_chain.invoke({'query':pergunta})
print(resposta['result'])

Hugging Face é uma empresa que iniciou em 2017 na França, inicialmente com o desenvolvimento de Chatbots. Atualmente, a empresa oferece uma plataforma de IA que disponibiliza modelos, conjuntos de dados e aplicativos chamados de "Spaces". Para acessar o Hugging Face, é necessário criar uma conta na plataforma e, para usar modelos restritos, é preciso solicitar autorização e criar um token de acesso.


### Map Reduce

In [16]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectorstore.as_retriever(search_type='mmr'),
    chain_type='map_reduce'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'

resposta = chat_chain.invoke({'query':pergunta})
print(resposta['result'])

Hugging Face é uma plataforma que oferece recursos e ferramentas para trabalhar com modelos de processamento de linguagem natural (NLP), como bibliotecas, modelos pré-treinados, entre outros. Para acessar o Hugging Face, você pode visitar o site oficial da empresa em https://huggingface.co/. Lá, você encontrará uma variedade de recursos, tutoriais e documentação para ajudá-lo a começar a trabalhar com NLP usando o Hugging Face. Além disso, é possível criar uma conta na plataforma para utilizar seus recursos, solicitar autorização para modelos restritos e gerar um token de acesso para utilizar os modelos disponíveis.


### Refine

In [17]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectorstore.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'

resposta = chat_chain.invoke({'query':pergunta})
print(resposta['result'])

Hugging Face é uma empresa que oferece uma plataforma de IA com modelos de linguagem pré-treinados, ferramentas e bibliotecas de Python. Para acessar a plataforma Hugging Face, você pode visitar o site oficial da empresa em https://huggingface.co/ e criar uma conta. Ao criar a conta, você poderá ter acesso a modelos de linguagem pré-treinados e outras funcionalidades oferecidas pela plataforma. Se você deseja usar modelos restritos, será necessário solicitar autorização de uso e criar um token de acesso, que permite que seu código entenda qual usuário está tentando usar o modelo e conceda acesso a ele. Assim, você poderá explorar todo o potencial da plataforma Hugging Face e utilizar seus recursos para desenvolver projetos de IA.
